# FSRS Parameter Divergence Exploration

This notebook explores the divergence between 'Nature' (the ground truth stability) and the FSRS model predictions under various scenarios. 

## Setup
We import the core simulation engine and optimized metric calculation logic.

In [1]:
import os
import sys
from collections import defaultdict
from datetime import timedelta

import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm

# Add src to path for imports
sys.path.append(os.path.abspath("../src"))

from simulate_fsrs import run_simulation
from simulation_config import SeededData, SimulationConfig
from utils import DEFAULT_PARAMETERS, calculate_metrics

T_EVAL = np.linspace(0, 100, 200)

In [2]:
# Shared configuration
REPEATS = int(os.getenv("REPEATS", "5"))
N_DAYS = int(os.getenv("N_DAYS", "180"))
SEED_HISTORY = os.getenv("ANKI_COLLECTION", "../collection.anki2")
DECK_CONFIG = os.getenv("DECK_CONFIG", "Default")

if SEED_HISTORY and not os.path.exists(SEED_HISTORY):
    SEED_HISTORY = None

## Helper Functions
Functions to run batches of simulations and plot the results.

In [3]:
def calculate_population_retrievability(t_eval, stabilities, params):
    import fsrs_engine

    # stabilities is an array of card stabilities
    # we calculate average retrievability at each time point in t_eval
    rets = []
    for t in t_eval:
        r = fsrs_engine.predict_retrievability(
            stabilities, np.full(len(stabilities), t), params
        )
        rets.append(np.mean(r))
    return np.array(rets)


def run_scenario(tasks):
    results = defaultdict(list)

    # Pre-load seeded payload for workers
    seeded_payload = None
    if SEED_HISTORY:
        from simulate_fsrs import _load_initial_state

        deck_t, deck_s, logs_data, current_date = _load_initial_state(
            DEFAULT_PARAMETERS,
            DEFAULT_PARAMETERS,
            None,
            SEED_HISTORY,
            DECK_CONFIG,
            None,
        )
        if not logs_data.is_empty:
            seeded_payload = SeededData(
                logs=logs_data,
                last_rev=current_date - timedelta(days=1),
                true_cards=deck_t,
                sys_cards=deck_s,
            )

    total_tasks = len(tasks) * REPEATS
    with tqdm(total=total_tasks, desc="Simulating") as pbar:
        for label, task_params in tasks.items():
            params = task_params.copy()
            if "n_days" not in params:
                params["n_days"] = N_DAYS

            if os.getenv("N_DAYS"):
                params["n_days"] = min(params["n_days"], int(os.getenv("N_DAYS")))

            for i in range(REPEATS):
                config = SimulationConfig(
                    seed=42 + i,
                    verbose=False,
                    compute_final_params=False,
                    return_logs=False,
                    **params,
                )
                try:
                    fitted, gt, metrics = run_simulation(
                        config=config, seeded_data=seeded_payload
                    )
                    if fitted:
                        s_nat, s_alg = metrics["stabilities"]
                        rmse, kl = calculate_metrics(gt, tuple(fitted), s_nat, s_alg)

                        r_nat = calculate_population_retrievability(T_EVAL, s_nat, gt)
                        r_fit = calculate_population_retrievability(
                            T_EVAL, s_alg, tuple(fitted)
                        )

                        results[label].append(
                            {"rmse": rmse, "kl": kl, "r_nat": r_nat, "r_fit": r_fit}
                        )
                except Exception as e:
                    print(f"Task {label} failed: {e}")
                pbar.update(1)

    return results


def plot_results(scenario_results, title):
    plt.figure(figsize=(12, 7))
    for label, repeats in scenario_results.items():
        if not repeats:
            continue
        avg_fit = np.mean([r["r_fit"] for r in repeats], axis=0)
        avg_nat = np.mean([r["r_nat"] for r in repeats], axis=0)
        avg_rmse = np.mean([r["rmse"] for r in repeats])
        avg_kl = np.mean([r["kl"] for r in repeats])

        (line,) = plt.plot(
            T_EVAL, avg_fit, label=f"{label} (RMSE: {avg_rmse:.4f}, KL: {avg_kl:.4f})"
        )
        plt.plot(T_EVAL, avg_nat, linestyle="--", color=line.get_color(), alpha=0.5)

    plt.title(title)
    plt.xlabel("Days since end of simulation")
    plt.ylabel("Aggregate Expected Retention")
    plt.legend(fontsize="small", ncol=1)
    plt.grid(True, alpha=0.3)
    plt.show()

## Scenario 1: Duration Exploration
Default parameters, 0.9 retention for different time periods (30, 90, 180, 365 days).

In [4]:
tasks_1 = {
    "30 Days": {"n_days": 30, "retention": "0.9"},
    "90 Days": {"n_days": 90, "retention": "0.9"},
    "180 Days": {"n_days": 180, "retention": "0.9"},
    "365 Days": {"n_days": 365, "retention": "0.9"},
}
res_1 = run_scenario(tasks_1)
plot_results(res_1, "Scenario 1: Impact of Simulation Duration (0.9 Retention)")

## Scenario 2: Retention Contrast
Default parameters, 0.9 vs 0.7 retention for the same time period (180 days).

In [5]:
tasks_2 = {
    "Ret=0.9": {"retention": "0.9"},
    "Ret=0.7": {"retention": "0.7"},
}
res_2 = run_scenario(tasks_2)
plot_results(res_2, "Scenario 2: Impact of Target Retention (180 Days)")

## Scenario 3: Static 0.9 vs Variable Schedule
Default params, static 0.9 vs a 5:1 weighted schedule favoring 0.9.

In [6]:
tasks_3 = {
    "Static 0.9": {"retention": "0.9"},
    "Sched 0.9:5,0.7:1": {"retention": "5:0.9,1:0.7"},
}
res_3 = run_scenario(tasks_3)
plot_results(res_3, "Scenario 3: Static 0.9 vs 0.9:5,0.7:1 Schedule")

## Scenario 4: Static 0.7 vs Variable Schedule
Default params, static 0.7 vs a 5:1 weighted schedule favoring 0.7.

In [7]:
tasks_4 = {
    "Static 0.7": {"retention": "0.7"},
    "Sched 0.7:5,0.9:1": {"retention": "5:0.7,1:0.9"},
}
res_4 = run_scenario(tasks_4)
plot_results(res_4, "Scenario 4: Static 0.7 vs 0.7:5,0.9:1 Schedule")

## Scenario 5: High-Retention Burn-In
Default params, 60 days of 0.9 burn-in followed by a shift to 0.7 vs staying at 0.9.

In [8]:
tasks_5 = {
    "0.9 BI -> 0.9 post": {"burn_in_days": 60, "retention": "0.9"},
    "0.9 BI -> 0.7 post": {
        "burn_in_days": 60,
        "retention": "60:0.9,120:0.7",
    },
}
res_5 = run_scenario(tasks_5)
plot_results(res_5, "Scenario 5: Post-Burn-In Retention Shift (0.9 Initial)")

## Scenario 6: Low-Retention Burn-In
Default params, 60 days of 0.7 burn-in followed by a shift to 0.9 vs staying at 0.7.

In [9]:
tasks_6 = {
    "0.7 BI -> 0.7 post": {"burn_in_days": 60, "retention": "0.7"},
    "0.7 BI -> 0.9 post": {
        "burn_in_days": 60,
        "retention": "60:0.7,120:0.9",
    },
}
res_6 = run_scenario(tasks_6)
plot_results(res_6, "Scenario 6: Post-Burn-In Retention Shift (0.7 Initial)")